# Importing and Exporting Template Matching Configurations

The `tt2DTM` package provides convenient program argument configurations through `.yaml` files.
These `.yaml` configuration files are essentially human-readable key-value pairs defining input arguments and their associated values.
For example, one argument is `micrograph_path` whose value should be a string to the path of the micrograph to be searched.
In the `.yaml` configuration file, this would look like

```yaml
...  # key-value pairs before
micrograph_path: "some/path/to/micrograph.mrc"
... # key-value pairs after
```

Validation of configurations and io is handled by Pydantic models which also provide a convenient way to interface with these arguments in a Python script.
Below, we go through the basics of creating a `.yaml` configuration for `tt2DTM`, parsing this into a `MatchTemplateManager` object, and re-exporting configurations.

> [!NOTE]
> The package is still under heavy development, so the layout and configuration arguments are subject to change in the future

First, we import the Pydantic models (under the submodule `tt2dtm.models`) used to keep track of and organize the information necessary for running template matching along with some other Python packages.

In [1]:
import torch

from tt2dtm.pydantic_models.computational_config import ComputationalConfig
from tt2dtm.pydantic_models.correlation_filters import (
    BandpassFilterConfig,
    PhaseRandomizationFilterConfig,
    PreprocessingFilters,
    WhiteningFilterConfig,
)
from tt2dtm.pydantic_models.defocus_search_config import DefocusSearchConfig
from tt2dtm.pydantic_models.match_template_manager import MatchTemplateManager
from tt2dtm.pydantic_models.match_template_result import MatchTemplateResult
from tt2dtm.pydantic_models.optics_group import OpticsGroup
from tt2dtm.pydantic_models.orientation_search_config import OrientationSearchConfig
from tt2dtm.pydantic_models.pixel_size_search_config import PixelSizeSearchConfig

## Hierarchical organization of match template configurations

The `MatchTemplateManager` class is the top-level object which contains the path to the micrograph, path to the 3D simulated volume of the reference template, and other configurations.
Each instance of the `MatchTemplateManager` class requires the following attributes during instantiation.
- `micrograph_path`: Path to the 2D image mrc file to be searched.
- `template_volume_path`: Path to the 3D volumetric mrc file for the reference template.
- `optics_group`: An instance of the `OpticsGroup` class, discussed below.
- `defocus_search_config`: An instance of the `DefocusSearchConfig` class, discussed below.
- `orientation_search_config`: An instance of the `OrientationSearchConfig` class, discussed below.
- `preprocessing_filters`: An instance of the `PreprocessingFilters` class, discussed below.
- `match_template_result`: An instance of the `MatchTemplateResult` class, discussed below.
- `computational_config`: An instance of the `ComputationalConfig` class, discussed below.

There are two additional attributes of the `MatchTemplateManager` class, `micrograph` and `template_volume`, which hold `torch.Tensor` objects of the loaded micrograph and template volume data, respectively.
Note that these attributes are automatically loaded from the mrc file paths, but the contents of these tensors are not serialized during model export.

As briefly mentioned above, when creating a new instance of a manager, the other configuration attributes are required.
We start with discussing the `OpticsGroup` class and work our way from the bottom up.

## The `OpticsGroup` class

The `OpticsGroup` class is a container for microscope imaging parameters used to calculate filters during the template matching process.

### Attributes

We follow the fields that are defined in [RELION's](https://relion.readthedocs.io/en/latest/) optics group .star file, and the class has the following attributes:
- `label`: A unique label for the optics group, usually contains some form of the micrograph name but can be any string.
- `pixel_size`: Float value representing the pixel size of the image, in Angstroms.
- `voltage`: The voltage of the microscope, in kV.
- `spherical_aberration`: The spherical aberration of the microscope, in mm, with the default value of 2.7 mm.
- `amplitude_contrast_ratio`: The amplitude contrast ratio, unitless, with the default value of 0.07.
- `phase_shift`: Additional phase shift to apply across the CTF, in degrees, with the default value of 0.0.
- `defocus_u`: Defocus of the micrograph along the major axis, in Angstroms.
- `defocus_v`: Defocus of the micrograph along the minor axis, in Angstroms.
- `astigmatism_angle`: Angle of the defocus astigmatism (relative to the x-axis), in degrees. The default value is 0.0.
- `ctf_B_factor`: An additional b-factor to apply to the CTF, in Angstroms^2. The default value is 0.0.

### Other unused attributes

Additional, currently unused attributes for modeling higher-order aberrations are also defined in the class.
These again follow RELION's conventions and may be used in the future.
For more information on these attributes, refer to our API documentation.


### An example of creating an `OpticsGroup` instance

Below we create an instance of the `OpticsGroup` class with some made up, but nevertheless realistic values.

In [2]:
optics_group = OpticsGroup(
    label="my_optics_group",
    pixel_size=1.06,
    voltage=300.0,
    spherical_aberration=2.7,  # default value
    amplitude_contrast_ratio=0.07,  # default value
    phase_shift=0.0,  # default value
    defocus_u=5200.0,
    defocus_v=4950.0,
    astigmatism_angle=25.0,
    ctf_B_factor=60.0,
)

### Serializing `OpticsGroup` instance to a dictionary

Pydantic has built-in functionality for generating a dictionary of key, value pairs from the model attributes and their values.
This can be done by calling the `.model_dump()` method for any of the Pydantic models in `tt2dtm.models`.

In [3]:
optics_group.model_dump()

{'label': 'my_optics_group',
 'pixel_size': 1.06,
 'voltage': 300.0,
 'spherical_aberration': 2.7,
 'amplitude_contrast_ratio': 0.07,
 'phase_shift': 0.0,
 'defocus_u': 5200.0,
 'defocus_v': 4950.0,
 'astigmatism_angle': 25.0,
 'ctf_B_factor': 60.0,
 'chromatic_aberration': 0.0,
 'mtf_reference': None,
 'mtf_values': None,
 'beam_tilt_x': None,
 'beam_tilt_y': None,
 'odd_zernike': None,
 'even_zernike': None,
 'zernike_moments': None}

### Exporting `OpticsGroup` instance to a `.yaml` file

[YAML](https://yaml.org) files are nothing more than a bunch of key-value pairs in a human-readable format.
Like [JSON](https://www.json.org), YAML has parser functions/libraries in most programming languages increasing their interoperability.
We adopt the `.yaml` format (and `.json` format, but not discussed here) for our configuration files rather than `.star` or some other custom serialization format.

While the other models could in theory have YAML export methods, 
To export the `OpticsGroup` instance (or any other Pydantic model we have in tt2dtm) to a `.yaml` file, we have the already implemented `to_yaml()` method.


In [4]:
yaml_filepath = "./optics_group_example.yaml"
optics_group.to_yaml(yaml_filepath)

The file `optics_group.yaml` should now exist within the same directory as this notebook and contains the following:

----

```yaml
amplitude_contrast_ratio: 0.07
beam_tilt_x: null
beam_tilt_y: null
chromatic_aberration: 0.0
ctf_B_factor: 60.0
astigmatism_angle: 25.0
defocus_u: 5200.0
defocus_v: 4950.0
even_zernike: null
label: my_optics_group
mtf_reference: null
mtf_values: null
odd_zernike: null
phase_shift: 0.0
pixel_size: 1.06
spherical_aberration: 2.7
voltage: 300.0
zernike_moments: null
```

----

### Importing `OpticsGroup` instance from a `.yaml` file

Each model also has the `from_yaml()` method which can be to instantiate the class from contents in a `.yaml` file.
Below, we are creating a new instance of the `OpticsGroup` class from the `optics_group.yaml` file.

In [5]:
new_optics_group = OpticsGroup.from_yaml(yaml_filepath)

# Check that attributes are the same
assert new_optics_group == optics_group

Editing, copying, and using `.yaml` files may be easier than directly manipulating objects in Python, especially when running large amounts of template matching jobs across many micrographs.
Check out the other example notebook and scripts for more ways to use these configurations in practice.

Below we continue with other model classes in the `tt2dtm.models` submodule, albeit in more brevity.

## The `DefocusSearchConfig` class

The `DefocusSearchConfig` class is a container for the parameters used for searching over defocus values during template matching.
It has the following attributes:
 - `enabled`: When true, do a defocus search during template matching.
 - `min_defocus`: The minimum defocus value to search over, in Angstroms.
 - `max_defocus`: The maximum defocus value to search over, in Angstroms.
 - `defocus_step`: The step size to search over the defocus values, in Angstroms.

 Note that the range of defocus values is relative to `defocus_u` and `defocus_v` in the `OpticsGroup` class, that is, we are searching above and below the average defocus value for the micrograph.

In [6]:
# This will produce searched (relative) defocus values of:
# [-600.0, -400.0, ..., 400.0, 600.0]  in Angstroms
defocus_search_config = DefocusSearchConfig(
    enable=True,
    defocus_min=-600.0,
    defocus_max=600.0,
    defocus_step=200.0,
)

defocus_search_config.model_dump()

{'enabled': True,
 'defocus_min': -600.0,
 'defocus_max': 600.0,
 'defocus_step': 200.0}

### The `DefocusSearchConfig.defocus_values` property

There is an additional helpful property, `defocus_values`, which is a list of defocus values to be searched over.
This list is not serialized.

In [7]:
defocus_search_config.defocus_values

[-600.0, -400.0, -200.0, 0.0, 200.0, 400.0, 600.0]

### The `PixelSizeSearchConfig` class

Nearly identical to the `DefocusSearchConfig` class, the `PixelSizeSearchConfig` contains minimum, maximum, and step size values for searching over pixel sizes.
Properties are:
- `enabled`: When true, do a pixel size search during template matching.
- `min_pixel_size`: The minimum pixel size to search over, in Angstroms.
- `max_pixel_size`: The maximum pixel size to search over, in Angstroms.
- `pixel_size_step`: The step size to search over the pixel sizes, in Angstroms.

In [8]:
pixel_size_search_config = PixelSizeSearchConfig(
    enable=True,
    pixel_size_min=1.02,
    pixel_size_max=1.11,
    pixel_size_step=0.02,
)
pixel_size_search_config.model_dump()

{'enabled': True,
 'pixel_size_min': 1.02,
 'pixel_size_max': 1.11,
 'pixel_size_step': 0.02}

### The `PixelSizeSearchConfig.pixel_sizes` property

There is an additional helpful property, `pixel_sizes`, which is a list of pixel sizes to be searched over.

In [9]:
pixel_size_search_config.pixel_sizes

[1.02, 1.04, 1.06, 1.08, 1.1, 1.12]

## The `OrientationSearchConfig` class

Container for orientation search parameters defining how SO(3) space is covered.
It has the following attributes:
 - `orientation_sampling_method`: String of SO(3) sampling method. Currently only supports "Hopf fibration".
 - `template_symmetry`: Symmetry group of template. Currently only supports "C1".
 - `psi_min`: Minimum psi angle to search over, in degrees. Default is 0.0.
 - `psi_max`: Maximum psi angle to search over, in degrees. Default is 360.0.
 - `theta_min`: Minimum theta angle to search over, in degrees. Default is 0.0.
 - `theta_max`: Maximum theta angle to search over, in degrees. Default is 180.0.
 - `phi_min`: Minimum phi angle to search over, in degrees. Default is 0.0.
 - `phi_max`: Maximum phi angle to search over, in degrees. Default is 360.0.
 - `in_plane_angular_step`: 
 - `out_of_plane_angular_step`: 

 Note that these parameters are used for generating Euler angles in the 'ZYZ' convention internally. Other orientation representation methods may be implemented in the future.

### Instantiation with default values

In [10]:
orientation_search_config = OrientationSearchConfig()
orientation_search_config.model_dump()

{'orientation_sampling_method': 'Hopf Fibration',
 'template_symmetry': 'C1',
 'psi_min': 0.0,
 'psi_max': 360.0,
 'theta_min': 0.0,
 'theta_max': 180.0,
 'phi_min': 0.0,
 'phi_max': 360.0,
 'in_plane_angular_step': 1.5,
 'out_of_plane_angular_step': 2.5}

### Non-default values

In [11]:
orientation_search_config = OrientationSearchConfig(
    orientation_sampling_method="Hopf Fibration",  # still default
    template_symmetry="C1",  # still default
    psi_min=120.0,
    psi_max=240.0,
    theta_min=45.0,
    theta_max=135.0,
    phi_min=20.0,
    phi_max=40.0,
    in_plane_angular_step=4.0,
    out_of_plane_angular_step=5.0,
)
orientation_search_config.model_dump()

{'orientation_sampling_method': 'Hopf Fibration',
 'template_symmetry': 'C1',
 'psi_min': 120.0,
 'psi_max': 240.0,
 'theta_min': 45.0,
 'theta_max': 135.0,
 'phi_min': 20.0,
 'phi_max': 40.0,
 'in_plane_angular_step': 4.0,
 'out_of_plane_angular_step': 5.0}

## The `PreprocessingFilters` class and subclasses

Multiple Fourier filters are calculated and used during template matching.
Rather than defining a singular model to hold all of the configurations, we have a base class, `PreprocessingFilters`, which holds instances of the following subclasses:
 - `PhaseRandomizationFilterConfig`
 - `WhiteningFilterConfig`
 - `BandpassFilterConfig`

Below are examples of creating instances of these classes and the `PreprocessingFilters` class.

In [12]:
# Enable phase randomization above 2.5 Angstroms
prf_config = PhaseRandomizationFilterConfig(enabled=False, cuton=2.5)
whitening_config = WhiteningFilterConfig(
    enabled=True,
    power_spectrum=True,
    smoothing=2.0,
)
bandpass_config = BandpassFilterConfig(
    enabled=False,  # filter not applied
    low_freq_cutoff=50.0,  # in Angstroms, converted to spatial frequency
    high_freq_cutoff=2.5,  # in Angstroms, converted to spatial frequency
    falloff=10.0,  # decay rate
)

# Place all filter configs into `PreprocessingFilters` object
preprocessing_filters = PreprocessingFilters(
    phase_randomization_filter_config=prf_config,
    whitening_filter_config=whitening_config,
    bandpass_filter_config=bandpass_config,
)
preprocessing_filters.model_dump()

{'whitening_filter_config': {'enabled': True,
  'power_spectrum': True,
  'smoothing': 2.0},
 'bandpass_filter_config': {'enabled': False,
  'low_freq_cutoff': 50.0,
  'high_freq_cutoff': 2.5,
  'falloff': 10.0},
 'phase_randomization_filter_config': {'enabled': False, 'cuton': 2.5}}

## The `ComputationalConfig` class

This class is currently unused, but its parameters may be used in the future to control how computational resources are used during template matching.

In [13]:
computational_config = ComputationalConfig(
    gpu_ids=[0, 1, 2, 3],
    num_cpus=20,
)
computational_config.model_dump()

{'gpu_ids': [0, 1, 2, 3], 'num_cpus': 20}

## The `MatchTemplateResult` class

Coming back to models which warrant more explanation of what's going on under-the-hood, we have the `MatchTemplateResult` class which holds parameters for where results should be saved.
In addition to the result paths, there is also the `allow_file_overwrite` attribute which will disallow overwriting files if set to `False`; the default is `False`.
There are also `torch.Tensor` attributes for each of the results, but these are not serialized.

In [14]:
match_template_result = MatchTemplateResult(
    allow_file_overwrite=True,
    mip_path="./output_mip.mrc",
    scaled_mip_path="./output_scaled_mip.mrc",
    correlation_average_path="./output_correlation_average.mrc",
    correlation_variance_path="./output_correlation_variance.mrc",
    orientation_psi_path="./output_orientation_psi.mrc",
    orientation_theta_path="./output_orientation_theta.mrc",
    orientation_phi_path="./output_orientation_phi.mrc",
    relative_defocus_path="./output_relative_defocus.mrc",
    pixel_size_path="./output_pixel_size.mrc",
)
match_template_result.model_dump()

{'allow_file_overwrite': True,
 'mip_path': './output_mip.mrc',
 'scaled_mip_path': './output_scaled_mip.mrc',
 'correlation_average_path': './output_correlation_average.mrc',
 'correlation_variance_path': './output_correlation_variance.mrc',
 'orientation_psi_path': './output_orientation_psi.mrc',
 'orientation_theta_path': './output_orientation_theta.mrc',
 'orientation_phi_path': './output_orientation_phi.mrc',
 'relative_defocus_path': './output_relative_defocus.mrc',
 'pixel_size_path': './output_pixel_size.mrc',
 'total_projections': 0,
 'total_orientations': 0,
 'total_defocus': 0}

### Helper method `MatchTemplateResult.export_results()`

When saving results, it becomes tedious to re-type or re-access paths for each result over and over again.
The `export_results()` method is a handy way to write all contents of the `MatchTemplateResult` instance to disk at once.

NOTE: Currently no header information is written to .mrc files! This will be added in the future but users beware, you must track a pixel's physical meaning for the time being.

In [15]:
# Setting the values to singleton tensors
match_template_result.mip = torch.Tensor([[1]])
match_template_result.scaled_mip = torch.Tensor([[1]])
match_template_result.correlation_average = torch.Tensor([[1]])
match_template_result.correlation_variance = torch.Tensor([[1]])
match_template_result.orientation_psi = torch.Tensor([[1]])
match_template_result.orientation_theta = torch.Tensor([[1]])
match_template_result.orientation_phi = torch.Tensor([[1]])
match_template_result.relative_defocus = torch.Tensor([[1]])
match_template_result.pixel_size = torch.Tensor([[1]])

# Export the dummy example files
match_template_result.export_results()

In [16]:
# Run this cell to remove the dummy example files after viewing
import os

os.remove("./output_mip.mrc")
os.remove("./output_scaled_mip.mrc")
os.remove("./output_correlation_average.mrc")
os.remove("./output_correlation_variance.mrc")
os.remove("./output_orientation_psi.mrc")
os.remove("./output_orientation_theta.mrc")
os.remove("./output_orientation_phi.mrc")
os.remove("./output_relative_defocus.mrc")
os.remove("./output_pixel_size.mrc")

## Returning to `MatchTemplateManager`

Now that the constituent parts of the `MatchTemplateManager` class have been discussed, we can instantiate a `MatchTemplateManager` object and export it to a `.yaml` file.
Note that currently the results must be separately saved to disk using the `MatchTemplateResult.export_results()` method, but there should be a way to integrate this into a single method call.

In [17]:
match_template_manager = MatchTemplateManager(
    micrograph_path="dummy_micrograph.mrc",
    template_volume_path="dummy_volume.mrc",
    optics_group=optics_group,
    defocus_search_config=defocus_search_config,
    orientation_search_config=orientation_search_config,
    pixel_size_search_config=pixel_size_search_config,
    preprocessing_filters=preprocessing_filters,
    match_template_result=match_template_result,
    computational_config=computational_config,
)

# This will display a lot of text
match_template_manager.model_dump()

{'micrograph_path': 'dummy_micrograph.mrc',
 'template_volume_path': 'dummy_volume.mrc',
 'optics_group': {'label': 'my_optics_group',
  'pixel_size': 1.06,
  'voltage': 300.0,
  'spherical_aberration': 2.7,
  'amplitude_contrast_ratio': 0.07,
  'phase_shift': 0.0,
  'defocus_u': 5200.0,
  'defocus_v': 4950.0,
  'astigmatism_angle': 25.0,
  'ctf_B_factor': 60.0,
  'chromatic_aberration': 0.0,
  'mtf_reference': None,
  'mtf_values': None,
  'beam_tilt_x': None,
  'beam_tilt_y': None,
  'odd_zernike': None,
  'even_zernike': None,
  'zernike_moments': None},
 'defocus_search_config': {'enabled': True,
  'defocus_min': -600.0,
  'defocus_max': 600.0,
  'defocus_step': 200.0},
 'orientation_search_config': {'orientation_sampling_method': 'Hopf Fibration',
  'template_symmetry': 'C1',
  'psi_min': 120.0,
  'psi_max': 240.0,
  'theta_min': 45.0,
  'theta_max': 135.0,
  'phi_min': 20.0,
  'phi_max': 40.0,
  'in_plane_angular_step': 4.0,
  'out_of_plane_angular_step': 5.0},
 'pixel_size_searc

Now that the `MatchTemplateManager` object has been created, we can export it to a `.yaml` file using the `to_yaml()` method.

In [18]:
match_template_manager.to_yaml("match_template_manager_example.yaml")

If you inspect the YAML file, you will see the following

----

```yaml
computational_config:
  gpu_ids:
  - 0
  - 1
  - 2
  - 3
  num_cpus: 20
defocus_search_config:
  defocus_max: 600.0
  defocus_min: -600.0
  defocus_step: 200.0
  enabled: true
match_template_result:
  allow_file_overwrite: true
  correlation_average_path: ./output_correlation_average.mrc
  correlation_variance_path: ./output_correlation_variance.mrc
  mip_path: ./output_mip.mrc
  orientation_phi_path: ./output_orientation_phi.mrc
  orientation_psi_path: ./output_orientation_psi.mrc
  orientation_theta_path: ./output_orientation_theta.mrc
  pixel_size_path: ./output_pixel_size.mrc
  relative_defocus_path: ./output_relative_defocus.mrc
  scaled_mip_path: ./output_scaled_mip.mrc
micrograph_path: dummy_micrograph.mrc
optics_group:
  amplitude_contrast_ratio: 0.07
  beam_tilt_x: null
  beam_tilt_y: null
  chromatic_aberration: 0.0
  ctf_B_factor: 60.0
  astigmatism_angle: 25.0
  defocus_u: 5200.0
  defocus_v: 4950.0
  even_zernike: null
  label: my_optics_group
  mtf_reference: null
  mtf_values: null
  odd_zernike: null
  phase_shift: 0.0
  pixel_size: 1.06
  spherical_aberration: 2.7
  voltage: 300.0
  zernike_moments: null
orientation_search_config:
  in_plane_angular_step: 4.0
  orientation_sampling_method: Hopf Fibration
  out_of_plane_angular_step: 5.0
  phi_max: 40.0
  phi_min: 20.0
  psi_max: 240.0
  psi_min: 120.0
  template_symmetry: C1
  theta_max: 135.0
  theta_min: 45.0
pixel_size_search_config:
  enabled: true
  pixel_size_max: 1.11
  pixel_size_min: 1.02
  pixel_size_step: 0.02
preprocessing_filters:
  bandpass_filter_config:
    enabled: false
    falloff: 10.0
    high_freq_cutoff: 2.5
    low_freq_cutoff: 50.0
  phase_randomization_filter_config:
    cuton: 2.5
    enabled: false
  whitening_filter_config:
    enabled: true
    power_spectrum: true
    smoothing: 2.0
template_volume_path: dummy_volume.mrc
```

----